### Linear Regression Analysis of the csr amount spent in a constituency

In [1]:
import pandas as pd
import numpy as np
import math
from statsmodels.formula.api import ols

In [2]:
def Log(x):
    ans = []
    for i in x:
        if i <= 0:
            ans.append(i)
        else: ans.append(math.log(i))
    return np.array(ans)
        

In [3]:
folder = '/home/manikaran/M.tech/Minor project/data/CSR_NEW_FILES'

In [4]:
dataDf = pd.read_csv('{}/{}/{}'.format(folder, '2014-15', 'dataForLinearRegressionAnalysis.csv'))

In [5]:
dataDf.shape

(542, 14)

In [6]:
dataDf.columns

Index(['Unnamed: 0', 'Constituency', 'state', 'Type', 'Margin %',
       'isMPMinister', 'centerAlignment', 'stateAlignment',
       'rs_min_mplad_district', 'is_backward', 'ministry_comp_intersection',
       'amount', 'non_backward', 'not_aligned'],
      dtype='object')

In [7]:
dataDf.rename({'Margin %':'Margin'}, inplace=True, axis=1)

In [8]:
dataDf.isna().any()

Unnamed: 0                    False
Constituency                  False
state                         False
Type                          False
Margin                        False
isMPMinister                  False
centerAlignment               False
stateAlignment                False
rs_min_mplad_district         False
is_backward                   False
ministry_comp_intersection    False
amount                        False
non_backward                  False
not_aligned                   False
dtype: bool

In [9]:
reqdCols = ['Type', 'Margin', 'isMPMinister', 'centerAlignment', 'stateAlignment', "not_aligned", 'rs_min_mplad_district', 'is_backward', 'non_backward', 'ministry_comp_intersection', 'amount']

In [10]:
df = dataDf[reqdCols]

In [11]:
df["amount"] = Log(df["amount"])

/home/manikaran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df = pd.concat([pd.get_dummies(df["Type"]), df], axis=1).drop("Type", axis=1)

In [31]:
df['aligned'] = df['stateAlignment'] | df['centerAlignment']

In [13]:
df.head()

,GEN,SC,ST,Margin,isMPMinister,centerAlignment,stateAlignment,not_aligned,rs_min_mplad_district,is_backward,non_backward,ministry_comp_intersection,amount
0,0,0,1,16.4,0.0,0,1,0,0,1,0,0,18.826146
1,0,1,0,28.1,1.0,1,0,0,0,0,1,0,20.809444
2,1,0,0,33.1,0.0,1,1,0,0,0,1,0,25.223409
3,0,1,0,33.2,0.0,1,1,0,0,0,1,0,25.205138
4,1,0,0,19.7,0.0,1,1,0,0,0,1,0,21.038077


In [39]:
model0 = ols("amount ~ aligned + not_aligned", df).fit()
model0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 amount   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.399
Date:                Tue, 23 Apr 2019   Prob (F-statistic):              0.248
Time:                        18:52:52   Log-Likelihood:                -1882.6
No. Observations:                 542   AIC:                             3771.
Df Residuals:                     539   BIC:                             3784.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -1.941e+12   1.22e+14     -0.016      0.987   -2.42e+14    2.38e+14
aligned      1.941e+12   1.22e+14      0.016      0.987   -2.38e+14    2.42e+14
not_aligned  1.941e+12   1.22e+14      0.016      0.987   -2.38e+14    2.42e+14
==============================================================================
Omnibus:                      160.978   Durbin-Watson:                   1.960
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              320.516
Skew:                          -1.735   Prob(JB):                     2.52e-70
Kurtosis:                       4.468   Cond. No.                     8.46e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.37e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [16]:
model1 = ols("amount ~ GEN:is_backward  + SC:is_backward + ST:is_backward + GEN:non_backward  + SC:non_backward + ST:non_backward", df).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 amount   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     2.720
Date:                Sun, 21 Apr 2019   Prob (F-statistic):             0.0194
Time:                        23:05:15   Log-Likelihood:                -1877.2
No. Observations:                 542   AIC:                             3766.
Df Residuals:                     536   BIC:                             3792.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           14.7482      0.467     31.606      0.000      13.832      15.665
GEN:is_backward      1.6075      0.835      1.926      0.055      -0.032       3.247
SC:is_backward      -0.4238      1.706     -0.248      0.804      -3.776       2.928
ST:is_backward       3.8648      1.419      2.723      0.007       1.077       6.653
GEN:non_backward     4.2580      0.593      7.181      0.000       3.093       5.423
SC:non_backward      3.6024      0.928      3.882      0.000       1.779       5.425
ST:non_backward      1.8393      1.446      1.272      0.204      -1.002       4.681
==============================================================================
Omnibus:                      161.934   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              323.948
Skew:                          -1.740   Prob(JB):                     4.52e-71
Kurtosis:                       4.496   Cond. No.                     3.24e+15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.47e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
model2 = ols("amount ~ GEN + GEN:is_backward + SC + SC:is_backward + ST + ST:is_backward", df).fit()
model2.summary()

In [ ]:
model3 = ols("amount ~ GEN + SC + ST + is_backward + non_backward", df).fit()
model3.summary()

In [ ]:
model3 = ols("amount ~ GEN + SC + ST + is_backward + non_backward", df).fit()
model3.summary()

In [17]:
model4 = ols("amount ~  GEN:is_backward  + SC:is_backward + ST:is_backward + GEN:non_backward  + SC:non_backward + ST:non_backward + Margin", df).fit()
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 amount   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     3.894
Date:                Sun, 21 Apr 2019   Prob (F-statistic):           0.000812
Time:                        23:05:50   Log-Likelihood:                -1872.4
No. Observations:                 542   AIC:                             3759.
Df Residuals:                     535   BIC:                             3789.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           13.5066      0.613     22.034      0.000      12.302      14.711
GEN:is_backward      1.5105      0.829      1.823      0.069      -0.117       3.138
SC:is_backward      -0.8174      1.698     -0.481      0.630      -4.153       2.518
ST:is_backward       3.7970      1.408      2.697      0.007       1.031       6.563
GEN:non_backward     3.9317      0.598      6.578      0.000       2.758       5.106
SC:non_backward      3.3115      0.925      3.578      0.000       1.494       5.130
ST:non_backward      1.7734      1.435      1.236      0.217      -1.046       4.593
Margin               0.0995      0.032      3.090      0.002       0.036       0.163
==============================================================================
Omnibus:                      156.491   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              305.913
Skew:                          -1.697   Prob(JB):                     3.73e-67
Kurtosis:                       4.425   Cond. No.                     6.18e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.78e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [18]:
model5 = ols("amount ~ stateAlignment + centerAlignment + not_aligned", df).fit()
model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 amount   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     4.051
Date:                Sun, 21 Apr 2019   Prob (F-statistic):            0.00728
Time:                        23:06:13   Log-Likelihood:                -1877.9
No. Observations:                 542   AIC:                             3764.
Df Residuals:                     538   BIC:                             3781.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          16.9386      1.090     15.537      0.000      14.797      19.080
stateAlignment      2.1049      0.857      2.455      0.014       0.421       3.789
centerAlignment     0.1611      0.872      0.185      0.854      -1.552       1.875
not_aligned        -0.6750      1.423     -0.474      0.636      -3.471       2.121
==============================================================================
Omnibus:                      161.965   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              323.825
Skew:                          -1.743   Prob(JB):                     4.81e-71
Kurtosis:                       4.481   Cond. No.                         7.87
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
model6 = ols("amount ~ stateAlignment + centerAlignment + not_aligned + isMPMinister + rs_min_mplad_district", df).fit()
model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 amount   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     4.526
Date:                Sun, 21 Apr 2019   Prob (F-statistic):           0.000474
Time:                        23:06:33   Log-Likelihood:                -1872.8
No. Observations:                 542   AIC:                             3758.
Df Residuals:                     536   BIC:                             3783.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                16.9565      1.082     15.670      0.000      14.831      19.082
stateAlignment            2.0516      0.851      2.410      0.016       0.380       3.724
centerAlignment           0.1514      0.885      0.171      0.864      -1.587       1.890
not_aligned              -0.7989      1.413     -0.565      0.572      -3.575       1.977
isMPMinister             -1.7598      1.177     -1.495      0.136      -4.073       0.553
rs_min_mplad_district     4.6952      1.652      2.842      0.005       1.449       7.941
==============================================================================
Omnibus:                      157.791   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              309.967
Skew:                          -1.709   Prob(JB):                     4.92e-68
Kurtosis:                       4.427   Cond. No.                         7.91
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
model7 = ols("amount ~ stateAlignment + centerAlignment + not_aligned + isMPMinister + rs_min_mplad_district + Margin", df).fit()
model7.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 amount   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                     5.203
Date:                Sun, 21 Apr 2019   Prob (F-statistic):           3.22e-05
Time:                        23:06:49   Log-Likelihood:                -1868.6
No. Observations:                 542   AIC:                             3751.
Df Residuals:                     535   BIC:                             3781.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                15.9568      1.130     14.126      0.000      13.738      18.176
stateAlignment            1.7609      0.851      2.068      0.039       0.088       3.434
centerAlignment          -0.2919      0.893     -0.327      0.744      -2.045       1.461
not_aligned              -0.6905      1.404     -0.492      0.623      -3.449       2.068
isMPMinister             -1.8870      1.170     -1.612      0.107      -4.186       0.412
rs_min_mplad_district     4.7713      1.641      2.907      0.004       1.547       7.996
Margin                    0.0956      0.033      2.878      0.004       0.030       0.161
==============================================================================
Omnibus:                      153.706   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              296.829
Skew:                          -1.676   Prob(JB):                     3.50e-65
Kurtosis:                       4.380   Cond. No.                         106.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
model8 = ols("amount ~ stateAlignment + centerAlignment + not_aligned + isMPMinister + rs_min_mplad_district + ministry_comp_intersection", df).fit()
model8.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 amount   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     6.361
Date:                Sun, 21 Apr 2019   Prob (F-statistic):           1.76e-06
Time:                        23:07:08   Log-Likelihood:                -1865.3
No. Observations:                 542   AIC:                             3745.
Df Residuals:                     535   BIC:                             3775.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     17.1220      1.069     16.015      0.000      15.022      19.222
stateAlignment                 1.8591      0.842      2.208      0.028       0.205       3.513
centerAlignment                0.0747      0.874      0.085      0.932      -1.642       1.792
not_aligned                   -0.8155      1.395     -0.585      0.559      -3.556       1.925
isMPMinister                  -5.2773      1.476     -3.575      0.000      -8.177      -2.378
rs_min_mplad_district          1.0919      1.879      0.581      0.561      -2.599       4.783
ministry_comp_intersection     7.2071      1.864      3.866      0.000       3.545      10.869
==============================================================================
Omnibus:                      155.438   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              303.042
Skew:                          -1.682   Prob(JB):                     1.57e-66
Kurtosis:                       4.450   Cond. No.                         10.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""